In [1]:
import pandas as pd
import os

In [2]:
def get_folders(dir: str = "") -> list:
    folders = os.listdir(f"{dir}/")
    return folders

In [3]:
metrics = ['MSETest','EnergyScoreTest', 'CRPSTest']

In [22]:
experiment = "darcy_flow"
model = "fno"
baseline = "scoring-rule-reparam"

In [23]:
def prepare_results(results_dir, experiment, model, baseline, metrics):
    results = pd.read_csv(f"{results_dir}/{experiment}/{model}/aggregated_results.csv", index_col=0)
    results = results.loc[metrics,results.loc["Statistic"] == "Mean"].astype(float)
    ref = results[[baseline]]
    results[["scoring-rule-dropout"]] = 1 - (results[["scoring-rule-dropout"]].values/ref.values)
    results[["scoring-rule-reparam"]] = 1 - (results[["scoring-rule-reparam"]].values/ref.values)
    results = results.transpose()
    results.insert(0, "Experiment", experiment)
    return results


In [38]:
results_dir = "../results/optimal_hp_multiple_seeds"
output_dir = "../results/skill_scores"
experiments = get_folders(dir = results_dir)
results = pd.DataFrame()
for experiment in experiments:
    model = "fno"
    if experiment == "sswe":
        model = "fno_1h_2"
    results = pd.concat([results,prepare_results(results_dir, experiment, model, baseline, metrics)], axis = 0)
results=results.round(4)
results.drop(["laplace", "dropout", "scoring-rule-reparam"], axis = 0, inplace = True)

In [39]:
results

,Experiment,MSETest,EnergyScoreTest,CRPSTest
scoring-rule-dropout,darcy_flow,0.2509,0.3030,0.3471
scoring-rule-dropout,era5,-0.0059,0.0704,0.0836
scoring-rule-dropout,ks,-0.0178,-0.0188,-0.1588
scoring-rule-dropout,sswe,0.2328,0.1910,0.4216


In [40]:
print(results.astype(str).to_latex())

\begin{tabular}{lllll}
\toprule
 & Experiment & MSETest & EnergyScoreTest & CRPSTest \\
\midrule
scoring-rule-dropout & darcy_flow & 0.2509 & 0.303 & 0.3471 \\
scoring-rule-dropout & era5 & -0.0059 & 0.0704 & 0.0836 \\
scoring-rule-dropout & ks & -0.0178 & -0.0188 & -0.1588 \\
scoring-rule-dropout & sswe & 0.2328 & 0.191 & 0.4216 \\
\bottomrule
\end{tabular}



In [42]:
# Dropout
results.drop("Experiment",axis = 1).loc['scoring-rule-dropout'].mean().round(4).values

array([0.115 , 0.1364, 0.1734])

In [41]:
# Reparam
results.drop("Experiment",axis = 1).loc['scoring-rule-reparam'].mean().round(4).values

KeyError: 'scoring-rule-reparam'